In [1]:
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
# Data Processing
import pandas as pd
import numpy as np
import requests

# Data Regru
import re

# Data Visualization
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import pyplot as plt
%matplotlib inline

# Machine Learning
from sklearn.metrics.pairwise import cosine_similarity 


###### to-do-list
1. server 어디를 이용할 지
2. back-up 자료를 풍성하게 준비
3. 군집화를 하고 customer's review 준비
    - 임의로 점수를 넣는 것보다 좋다 안좋다로 하는 것이 좋을 수 있다.
    - 데이터 줄이기 (약 3000개)  (이미지 없는거 빼기)

# Purpose 
**Here's what wine you should be drinking, according to how you take your coffee.**

- https://www.businessinsider.com/wine-reccomendations-based-on-coffee-2015-9
- https://www.google.com/search?q=NY+Mulino+a+Vino&oq=NY+Mulino+a+Vino&aqs=chrome..69i57j0i7i8i30.3828j0j9&sourceid=chrome&ie=UTF-8

# Data crawling 
- data crawled from www.wine21.com
- price : under ₩ 200,000
- wine types limited to red, white, rose, sparkling and porto
- use only items being sold in off-line stores like E-mart, Lotte, Costco, and etc.

In [3]:
df_1 = pd.read_csv('./data/df_aroma.csv')
df_1.drop(['Unnamed: 0'], axis=1, inplace=True)
df_1.tail(2)

,title,image_link,winery,country,province,grape_type,wine_type,occasion,alcohol,sweetness,acidity,body_rate,tannin_rate,food_pairing,price,vintage,size,link,aroma_y
1908,칭퀘테레 코스타 데 캄푸,http://img.wine21.com/WINE_MST/TITLE/0154000/W...,깐티나 칭퀘테레,이탈리아,리구리아,"['보스코', '알라롤라', '베르멘티노']",화이트,테이블 와인,8.5,1,3,3,1,"신선한 샐러드, 각종 파스타류 또는 풍미가 강하지 않은 생선과 잘 어울린다.",80000,2013,750,https://www.wine21.com/13_search/wine_view.htm...,"금빛 반짝이는 노란색을 띠고, 은은히 느껴지는 세이지, 허브 향과 시트러스, 야생 ..."
1909,"깐티나 칭퀘테레, 칭퀘테레",http://img.wine21.com/WINE_MST/TITLE/0154000/W...,깐티나 칭퀘테레,이탈리아,리구리아,"['보스코', '알라롤라', '베르멘티노']",화이트,테이블 와인,8.5,1,3,4,1,"전채요리, 차가운 육류요리, 각종 시푸드 및 생선요리와 완벽한 조화를 이룬다.",60000,2013,750,https://www.wine21.com/13_search/wine_view.htm...,"영롱하게 반짝이는 옅은 황금 노란 빛을 띠고, 강렬하면서도 부드러운 야생 꽃 과 꿀..."


# Data Processing

In [4]:
df = df_1.copy()

## grape_type : use the first species

In [5]:
for i in range(len(df)):
    if ',' in df['grape_type'][i]:
        df['grape_type'][i] = df['grape_type'][i][2:-2].split(',')[0][:-1]
    else:
        df['grape_type'][i] = df['grape_type'][i][2:-2]
df['grape_type']

0        바르베라
1        바르베라
2        샤르도네
3       피노 누아
4       피노 누아
        ...  
1905     샤르도네
1906      보스코
1907      보스코
1908      보스코
1909      보스코
Name: grape_type, Length: 1910, dtype: object

In [6]:
df['grape_type'].unique()

array(['바르베라', '샤르도네', '피노 누아', '카베르네 소비뇽', '메를로', '마르산느', '산지오베제',
       '비오시뉴', '투리가 프란체사', '시라/쉬라즈', '말벡', '슈냉 블랑', '진판델', '모나스트렐',
       '소비뇽 블랑', '그르나슈', '쌩쏘', '모스카토', '블랜드', '템프라니요', '네로 다볼라', '프리미티보',
       '네비올로', '카르메네르', '투리가 나시오날', '자렐로', '베르데호', '아시르티코', '시노마브로',
       '마브루드', '아기오르기티코', '로디티스', '빌라나', '까리냥', '모스카텔 세투발', '가르나차', '피아노',
       '마카베오', '틴타 네그라 몰레', '아이렌', '리슬링', '글레라', '오미자', '코르비나 베로네제',
       '쁘띠 시라', '누라구스', '소비뇽 그리', '베르디키오 비앙코', '몬테풀치아노', '라크리마', '비오니에',
       '모스카텔', '푸르민트', '젤렌', '트라하두라', '그르나슈 블랑', '멘시아', '보발', '돌체토',
       '코르테제', '아르네이스', '뮈스까 까넬리', '세르시알', '알리아니코', '피노 네로', '세미용',
       '베르멘티노', '카베르네 프랑', '믈롱 드 부르고뉴', '피노 그리지오', '트라미너', '비달', '피노 뮈니에',
       '가르나차 틴토레라', '레불라', '쁘띠 메슬리에', '모작', '네로 디 트로이아', '레포스코', '투르비아노',
       '말바시아 디 칸디다', '클레렛', '피노 그리', '페르나오 피레스', '아라고네즈', '코르비나', '론디넬라',
       '롤', '카스텔라옹', '비칼', '사그란티노', '고데요', '트레비아노', '알바리뇨', '플라바치 말리',
       '포십', '게뷔르츠트라미너', '모스까뗄 데 알레한드리아', '오르테가', '틴타 로리즈', '청포도 즙',
    

In [7]:
# dropped 'wine' made from apple
df.drop(index=df[df['grape_type']=='사과'].index, inplace=True)
df.reset_index(inplace=True)

## one_hot encoding

### food_pairing : one_hot encoding
- add meat / cheese / seafood / vegetables / dessert columns
- meat : 고기, 육류, 돼지, 소, 비프,  닭, 오리, 양, 스테이크, 보쌈, 냉채, 바비큐, 하몽, 불고기, 햄, 쇠고기, 소시지, 칠면조, 치킨,
- cheese : 치즈
- seafood : 해산물, 조개, 새우, 갑각류, 연어, 참치, 초밥, 생선, 굴, 회, 사시미, 관자,
- vegetables : 버섯, 딸기, 올리브, 아몬드, 샐러드, 아보카도, 과일, 호두, 멜론, 무화과, 야채, 견과류, 
- dessert : 디저트, 수플레, 케익, 케이크, 파이, 마카롱, 아이스크림,

In [8]:
meat_ls = ['고기', '육류', '돼지', '소', '비프', '닭', '오리', '양', '스테이크', '보쌈', '냉채', '바비큐', 
           '하몽', '불고기', '햄', '쇠고기', '소시지', '칠면조', '치킨']
cheese_ls = ['치즈']
seafood_ls = ['해산물', '조개', '새우', '갑각류', '연어', '참치', '초밥', '생선', '굴', '회', '사시미', '관자']
vegetables_ls = ['버섯', '딸기', '올리브', '아몬드', '샐러드', '아보카도', '과일', '호두', '멜론', '무화과', '야채', '견과류']
dessert_ls = ['디저트', '수플레', '케익', '케이크', '파이', '마카롱', '아이스크림']

In [9]:
df['food_pairing']

0       피자, 파스타,  붉은 육류,  그리고 치즈 플레이트와 잘 어울리며, 특유의 산도가...
1                    식전 와인으로 적합하며 전체요리나 기름진 생선요리와 잘 어울린다.
2                                 치즈, 해산물, 샐러드 등과 잘 어울린다.
3                                     치즈, 오리고기 등과 잘 어울린다.
4                        치즈, BBQ, 구운 고기, 버섯 요리 등과 잘 어울린다.
                              ...                        
1904    너무 차지 않게 해 식전주로 마시거나 살구 타르트 등의 과일 디저트와 곁들이면 잘 ...
1905             와인 자체로도 완벽하지만, 각종 디져트와 향이 강한 치즈와 잘 어울린다.
1906         신선한 셀러드, 각종 파스타류 또는 풍미가 강하지 않은 생선과 잘 어울린다.\n
1907           신선한 샐러드, 각종 파스타류 또는 풍미가 강하지 않은 생선과 잘 어울린다.
1908          전채요리, 차가운 육류요리, 각종 시푸드 및 생선요리와 완벽한 조화를 이룬다.
Name: food_pairing, Length: 1909, dtype: object

In [10]:
# add meat category column
df['meat'] = 0
df['cheese'] = 0
df['seafood'] = 0
df['vegetables'] = 0
df['dessert'] = 0

for i in range(len(df['food_pairing'])):
    for meat in meat_ls:
        if meat in df['food_pairing'][i]:
            df['meat'][i] = 1
    for cheese in cheese_ls:
        if cheese in df['food_pairing'][i]:
            df['cheese'][i] = 1
    for seafood in seafood_ls:
        if seafood in df['food_pairing'][i]:
            df['seafood'][i] = 1
    for vegetables in vegetables_ls:
        if vegetables in df['food_pairing'][i]:
            df['vegetables'][i] = 1
    for dessert in dessert_ls:
        if dessert in df['food_pairing'][i]:
            df['dessert'][i] = 1

In [11]:
df[['food_pairing', 'meat', 'cheese', 'seafood', 'vegetables', 'dessert']]

,food_pairing,meat,cheese,seafood,vegetables,dessert
0,"피자, 파스타, 붉은 육류, 그리고 치즈 플레이트와 잘 어울리며, 특유의 산도가...",1,1,0,0,0
1,식전 와인으로 적합하며 전체요리나 기름진 생선요리와 잘 어울린다.,0,0,1,0,0
2,"치즈, 해산물, 샐러드 등과 잘 어울린다.",0,1,1,1,0
3,"치즈, 오리고기 등과 잘 어울린다.",1,1,0,0,0
4,"치즈, BBQ, 구운 고기, 버섯 요리 등과 잘 어울린다.",1,1,0,1,0
...,...,...,...,...,...,...
1904,너무 차지 않게 해 식전주로 마시거나 살구 타르트 등의 과일 디저트와 곁들이면 잘 ...,0,0,0,1,1
1905,"와인 자체로도 완벽하지만, 각종 디져트와 향이 강한 치즈와 잘 어울린다.",0,1,0,0,0
1906,"신선한 셀러드, 각종 파스타류 또는 풍미가 강하지 않은 생선과 잘 어울린다.\n",0,0,1,0,0
1907,"신선한 샐러드, 각종 파스타류 또는 풍미가 강하지 않은 생선과 잘 어울린다.",0,0,1,1,0


### aroma categories
- append aroma category dummies based on string values in aroma_y column
- floral / fruit / cologne / oriental_leather / oriental_spice / earth*
* aroma criteria based off of wine aroma wheel from
https://www.kellyandjones.com/blogs/news/perfumers-guide-to-wine-aromas

In [12]:
floral = ['로럴', '백합', '자스민', '제라니움', '쟈스민', '피오니', '꽃', '아이리스', '부케', '아카시아', '장미', '클로버', '로즈힙',
         '플라워', '카모마일']
fruit = ['트로피칼', '복숭아', '과일','과육', '열매', '스트로베리', '딸기', '블루베리', '베리', '아로니아', '과실', '라스베리', '자두', '체리', 
         '카시스', '사과', '배향', '커런트', '멜론', '애플', '프룻', '매실', '프루티', '살구', '애플리콧', '배', '무화과', '플럼', 
         '패션후르츠', '오디', '모과', '파파야', '망고', '석류', '버찌', '코코넛', '바나나', '호박']
citrus = ['시트러스', '씨트런향', '자몽', '오렌지', '레몬', '귤', '라임', '탠저린', '베르가못']

oriental_leather = ['에스프레소', '담배', '가죽', '쵸콜릿', '바닐라', '카라멜', '초콜릿', '캬라멜', '스모크', '흑연', '모카', 
                    '스파이', '꿀',  '헤이즐넛', '코코아', '타바코', '시가', '레더', '아몬드', '호두','허니', '크림']
oriental_spice = ['향신료', '세이지', '감초', '씨나몬', '라벤다', '라벤더', '발사믹','스파이스' '스파이시', '계피', '정향', '페퍼', 
                  '후추']
earth = ['낙엽', '흙내', '커피', '삼나무', '로즈마리', '나무', '버섯', '허브', '유칼립투스', '토양', '밀짚','회향', '젖은 돌', 
         '머쉬룸', '트러플']


In [13]:
# convert data type from object to string type data

df.aroma_y = df.aroma_y.apply(lambda x: str(x))

In [14]:
# add floral category column

df['floral'] = 0
df['fruit'] = 0
df['citrus'] = 0
df['oriental_spice'] = 0
df['oriental_leather'] = 0
df['earth'] = 0

for i in range(len(df)):
    for floral_ in floral:
        if floral_ in df['aroma_y'][i]:
            df['floral'][i] = 1
    for fruit_ in fruit:
        if fruit_ in df['aroma_y'][i]:
            df['fruit'][i] = 1
    for citrus_ in citrus:
        if citrus_ in df['aroma_y'][i]:
            df['citrus'][i] = 1
    for oriental_spice_ in oriental_spice:
        if oriental_spice_ in df['aroma_y'][i]:
            df['oriental_spice'][i] = 1
    for oriental_leather_ in oriental_leather:
        if oriental_leather_ in df['aroma_y'][i]:
            df['oriental_leather'][i] = 1
    for earth_ in earth:
        if earth_ in df['aroma_y'][i]:
            df['earth'][i] = 1

In [15]:
for name in ['floral', 'fruit', 'citrus', 'oriental_spice', 'oriental_leather','earth']:
    print(df[name].value_counts())

0    1452
1     457
Name: floral, dtype: int64
1    1687
0     222
Name: fruit, dtype: int64
0    1508
1     401
Name: citrus, dtype: int64
0    1497
1     412
Name: oriental_spice, dtype: int64
0    1138
1     771
Name: oriental_leather, dtype: int64
0    1477
1     432
Name: earth, dtype: int64


#### dark/fruity aroma
- aroma category combinations created for filtering wine recommendation list based on coffee type preference

In [16]:
# fruity aroma : floral, fruit, citrus

df[((df['floral'] ==1) | (df['fruit'] == 1) | (df['citrus'] == 1)) & (df['oriental_spice'] ==0) & 
   (df['oriental_leather'] ==0) & (df['earth'] ==0)]

,index,title,image_link,winery,country,province,grape_type,wine_type,occasion,alcohol,...,cheese,seafood,vegetables,dessert,floral,fruit,citrus,oriental_spice,oriental_leather,earth
0,0,트리풀라 피에몬테 로쏘,http://img.wine21.com/WINE_MST/TITLE/0167000/W...,몬도 델 비노,이탈리아,피에몬테,바르베라,레드,테이블 와인,13.5,...,1,0,0,0,1,1,0,0,0,0
1,1,아케시 로사토 브뤼,http://img.wine21.com/WINE_MST/TITLE/0167000/W...,몬도 델 비노,이탈리아,피에몬테,바르베라,스파클링,테이블 와인,11.5,...,0,1,0,0,1,1,0,0,0,0
2,2,엘리자베스 로제 샤도네이,http://img.wine21.com/WINE_MST/TITLE/0167000/W...,고스트 블락 와이너리,미국,캘리포니아,샤르도네,화이트,테이블 와인,13.5,...,1,1,1,0,0,1,1,0,0,0
5,5,"백하우스, 샤도네이",http://img.wine21.com/WINE_MST/TITLE/0167000/W...,백하우스,미국,캘리포니아,샤르도네,화이트,테이블 와인,13.5,...,0,1,1,0,0,1,1,0,0,0
7,7,"올리비에 르플레브, 부르고뉴 “옹끌 뱅썽”",http://img.wine21.com/WINE_MST/TITLE/0167000/W...,올리비에 르플레브,프랑스,부르고뉴,샤르도네,화이트,테이블 와인,13.5,...,0,1,0,0,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1897,1898,글라디움 템프라니요,http://img.wine21.com/WINE_MST/TITLE/0154000/1...,보데가스 깜뽀스 레알레스,스페인,라,템프라니요,레드,테이블 와인,8.5,...,1,0,1,0,0,1,0,0,0,0
1899,1900,글라디움 아이렌,http://img.wine21.com/WINE_MST/TITLE/0154000/1...,보데가스 깜뽀스 레알레스,스페인,라,아이렌,화이트,테이블 와인,8.0,...,0,1,0,0,0,1,0,0,0,0
1901,1902,"떼누따 델레 떼레, 에뜨나 비앙코",http://img.wine21.com/WINE_MST/TITLE/0154000/1...,떼누따 델레 떼레 네레,이탈리아,시칠리아,카리칸테,화이트,테이블 와인,13.0,...,0,1,0,0,1,0,0,0,0,0
1904,1905,"샹파뉴 델라모뜨, 브뤼",http://img.wine21.com/WINE_MST/TITLE/0154000/W...,샹파뉴 들라모뜨,프랑스,샹파뉴,샤르도네,스파클링,테이블 와인,12.0,...,0,0,1,1,1,1,1,0,0,0


In [17]:
# dark aroma : oriental_spice, oriental_leather, earth

df[((df['oriental_spice'] == 1) | (df['oriental_leather'] == 1) | (df['earth'] ==1)) & 
  (df['floral'] == 0) & (df['fruit'] == 0) & (df['citrus'] == 0)]

,index,title,image_link,winery,country,province,grape_type,wine_type,occasion,alcohol,...,cheese,seafood,vegetables,dessert,floral,fruit,citrus,oriental_spice,oriental_leather,earth
139,139,"아나이, 파운더스 콜렉션 시라",http://img.wine21.com/WINE_MST/TITLE/0167000/W...,아나이,칠레,라펠,시라/쉬라즈,레드,테이블 와인,13.5,...,0,0,0,0,0,0,0,1,0,0
149,149,리저브 생 마르땡 까베르네 소비뇽,http://img.wine21.com/WINE_MST/TITLE/0167000/W...,리저브 생 마르땡,프랑스,서던,카베르네 소비뇽,레드,테이블 와인,8.5,...,0,0,0,0,0,0,0,1,0,1
294,294,"에구렌 우가르떼, 크리안자",http://img.wine21.com/WINE_MST/TITLE/0166000/W...,에구렌 우가르떼,스페인,리오하,템프라니요,레드,테이블 와인,13.5,...,0,1,0,0,0,0,0,0,1,1
295,295,"에구렌 우가르떼, 코세차",http://img.wine21.com/WINE_MST/TITLE/0166000/W...,에구렌 우가르떼,스페인,리오하,템프라니요,레드,테이블 와인,13.5,...,0,1,0,0,0,0,0,0,1,1
296,296,"에구렌 우가르떼, 레세르바",http://img.wine21.com/WINE_MST/TITLE/0166000/W...,에구렌 우가르떼,스페인,리오하,템프라니요,레드,테이블 와인,14.5,...,0,1,0,0,0,0,0,0,1,1
320,320,"리버비, 싱글 빈야드 샤르도네",http://img.wine21.com/WINE_MST/TITLE/0166000/W...,리버비 에스테이트,뉴질랜드,사우스,샤르도네,화이트,테이블 와인,12.5,...,0,0,0,0,0,0,0,0,0,1
447,447,"로랑 페리에, 하모니 드미 섹",NaN,로랑 페리에,프랑스,샹파뉴,샤르도네,스파클링,테이블 와인,12.5,...,0,0,0,1,0,0,0,0,1,0
584,584,"도멘 드 라 호예르, 루베롱 블랑",http://img.wine21.com/WINE_MST/TITLE/0164000/W...,도멘 드 라 호예르,프랑스,론,그르나슈 블랑,화이트,테이블 와인,13.5,...,1,1,1,0,0,0,0,0,1,1
685,685,오르넬로,http://img.wine21.com/WINE_MST/TITLE/0163000/W...,까스텔라레 디 까스텔리나,이탈리아,토스카나,산지오베제,레드,테이블 와인,14.5,...,1,0,0,0,0,0,0,0,1,0
715,715,"테사리, 아브스",http://img.wine21.com/WINE_MST/TITLE/0163000/0...,테사리,이탈리아,베네토,가르가네가,화이트,테이블 와인,12.5,...,0,1,0,0,0,0,0,0,1,1


In [18]:
df['dark_aroma'] = 0
df['fruity_aroma'] = 0

for i in df[(df.oriental_spice==1) | (df.oriental_leather==1) | (df.earth==1)].index:
    df['dark_aroma'][i] = 1

for i in df[(df.floral==1) | (df.fruit==1) | (df.citrus==1)].index:
    df['fruity_aroma'][i] = 1

In [19]:
df[['dark_aroma', 'fruity_aroma']]

,dark_aroma,fruity_aroma
0,0,1
1,0,1
2,0,1
3,1,1
4,1,1
...,...,...
1904,0,1
1905,0,1
1906,1,1
1907,1,1


In [20]:
# drop wines which weren't categorized by aroma
print('wines without aroma category : ', len(df[(df['dark_aroma'] == 0) & (df['fruity_aroma'] == 0)]))

df = df[(df['dark_aroma'] == 1) | (df['fruity_aroma'] == 1)]
df.reset_index(inplace=True)

print('data count : ', len(df))

wines without aroma category :  134
data count :  1775


### continent
- grouping countries based on their geographic location and climate
- necessary to reduce bias due to overfitting

- Europe_A : 프랑스, 오스트리아, 몰도바, 독일, 헝가리, 슬로베니아, 불가리아, 조지아, 루마니아
- Europe_B : 이탈리아, 포르투갈, 스페인, 그리스, 크로아티아, 이스라엘, 레바논
- North_America : 미국, 캐나다
- South_America : 아르헨티나, 칠레, 우루과이
- New_World : 호주, 뉴질랜드, 남아프리카
- Korea : 대한민국

In [21]:
europe_a = ['프랑스', '오스트리아', '몰도바', '독일', '헝가리', '슬로베니아', '불가리아', '조지아', '루마니아']
europe_b = ['이탈리아', '포르투갈', '스페인', '그리스', '크로아티아', '이스라엘', '레바논']
north_america = ['미국', '캐나다']
south_america = ['아르헨티나', '칠레', '우루과이'] 
new_world = ['호주', '뉴질랜드', '남아프리카']
korea = ['대한민국']

In [22]:
df['europe_a'] = 0
df['europe_b'] = 0
df['north_america'] = 0
df['south_america'] = 0
df['new_world'] = 0
df['korea'] = 0

for i in range(len(df['country'])):
    for a in europe_a:
        if a in df['country'][i]:
            df['europe_a'][i] = 1
    for b in europe_b:
        if b in df['country'][i]:
            df['europe_b'][i] = 1
    for north in north_america:
        if north in df['country'][i]:
            df['north_america'][i] = 1
    for south in south_america:
        if south in df['country'][i]:
            df['south_america'][i] = 1
    for new in new_world:
        if new in df['country'][i]:
            df['new_world'][i] = 1
    for k in korea:
        if k in df['country'][i]:
            df['korea'][i] = 1
            
df[['country', 'europe_a', 'europe_b', 'north_america', 'south_america', 'new_world', 'korea']]

,country,europe_a,europe_b,north_america,south_america,new_world,korea
0,이탈리아,0,1,0,0,0,0
1,이탈리아,0,1,0,0,0,0
2,미국,0,0,1,0,0,0
3,미국,0,0,1,0,0,0
4,미국,0,0,1,0,0,0
...,...,...,...,...,...,...,...
1770,프랑스,1,0,0,0,0,0
1771,이탈리아,0,1,0,0,0,0
1772,이탈리아,0,1,0,0,0,0
1773,이탈리아,0,1,0,0,0,0


In [23]:
# count european wines
print('europe_a wine_count : ', len(df[(df['europe_a'] == 1)]))
print('europe_b wine_count : ', len(df[(df['europe_b'] == 1)]))
print('north_america wine_count : ', len(df[(df['north_america'] == 1)]))
print('south_america wine_count : ', len(df[(df['south_america'] == 1)]))
print('new_world wine_count : ', len(df[(df['new_world'] == 1)]))
print('korea wine_count : ', len(df[(df['korea'] == 1)]))

europe_a wine_count :  383
europe_b wine_count :  728
north_america wine_count :  244
south_america wine_count :  229
new_world wine_count :  189
korea wine_count :  2


## wine_type 
- '로제' & '스파클링' -> '화이트'
- '주정강화' -> '레드' or '화이트'
    - port wine -> '레드'
    - the others -> '삭제'

In [24]:
df['wine_type'].unique()

array(['레드', '스파클링', '화이트', '주정강화', '로제'], dtype=object)

In [25]:
df[(df['wine_type'] == '로제') | (df['wine_type'] == '스파클링')]['wine_type'] = '화이트'

In [26]:
for i in range(len(df)):
    if '포트' in df['title'][i]:
        df['wine_type'][i] = '레드'

In [27]:
print("Red Counts : " , round((len(df[df['wine_type'] == '레드']) / len(df)) * 100, 2), "%")
print("White Counts : " , round((len(df[df['wine_type'] == '화이트']) / len(df)) * 100, 2), "%")

Red Counts :  57.63 %
White Counts :  27.21 %


In [28]:
for i in range(len(df)):
    if df['wine_type'][i] == '레드':
        df['wine_type'][i] = 'red'
    elif df['wine_type'][i] == '화이트':
        df['wine_type'][i] = 'white'
    else:
        df['wine_type'][i] = 'etc'

In [29]:
# delect etc(sherry wine)
for i in range(len(df)):
    if df['wine_type'][i] == 'etc':
        df2 = df.drop([i], inplace=True)

In [30]:
len(df)

1506

In [31]:
# one_hot encdoing for wine_type
df[['red', 'white']] = 0
df[['red', 'white']] = pd.get_dummies(df['wine_type'])

In [32]:
print("Red Counts : " , round((len(df[df['wine_type'] == 'red']) / len(df)) * 100, 2), "%")
print("White Counts : " , round((len(df[df['wine_type'] == 'white']) / len(df)) * 100, 2), "%")

Red Counts :  67.93 %
White Counts :  32.07 %


In [33]:
df.columns

Index(['level_0', 'index', 'title', 'image_link', 'winery', 'country',
       'province', 'grape_type', 'wine_type', 'occasion', 'alcohol',
       'sweetness', 'acidity', 'body_rate', 'tannin_rate', 'food_pairing',
       'price', 'vintage', 'size', 'link', 'aroma_y', 'meat', 'cheese',
       'seafood', 'vegetables', 'dessert', 'floral', 'fruit', 'citrus',
       'oriental_spice', 'oriental_leather', 'earth', 'dark_aroma',
       'fruity_aroma', 'europe_a', 'europe_b', 'north_america',
       'south_america', 'new_world', 'korea', 'red', 'white'],
      dtype='object')

## data reduction

In [34]:
# drop unnessary columns
df.drop('occasion', axis=1, inplace=True)
df.drop('index', axis=1, inplace=True)
df.drop('level_0', axis=1, inplace=True)

In [35]:
# delete tesco wines
df[(df['winery'] == '테스코 파이니스트') | (df['winery'] == '테스코 그룹  (테스코 심플리 와인) ')] = \
df[(df['winery'] == '테스코 파이니스트') | (df['winery'] == '테스코 그룹  (테스코 심플리 와인) ')].drop(index=df[(df['winery'] == '테스코 파이니스트') | (df['winery'] == '테스코 그룹  (테스코 심플리 와인) ')].index
, inplace=True)

In [36]:
# delete wine's size either over or below than 750ml
df = df[df['size'] == 750.0]

In [37]:
# delete image(NAN)

df = df[df.image_link.notnull()]

In [38]:
df.reset_index(drop=True, inplace=True)
df['wine_id'] = df.index

In [39]:
df.isnull().sum()

title                0
image_link           0
winery               0
country              0
province            63
grape_type           0
wine_type            0
alcohol              0
sweetness            0
acidity              0
body_rate            0
tannin_rate          0
food_pairing         0
price                0
vintage             15
size                 0
link                 0
aroma_y              0
meat                 0
cheese               0
seafood              0
vegetables           0
dessert              0
floral               0
fruit                0
citrus               0
oriental_spice       0
oriental_leather     0
earth                0
dark_aroma           0
fruity_aroma         0
europe_a             0
europe_b             0
north_america        0
south_america        0
new_world            0
korea                0
red                  0
white                0
wine_id              0
dtype: int64

In [40]:
df.to_csv('df_aroma_processing.csv', encoding='utf-8-sig')

# Questionnaire for service user

###### Survey

> 1) Which coffee do you prefer to drink?
1. black coffee (e.g. americano, espresso)
2. coffee with milk & sugar (eg. frappuccino, vanila latte, etc.)
3. I prefer tea

> 2) Which type of coffee bean (or tea) do you prefer?
1. acidic type
2. full body type

###### cases (1000 cases per type)
- dark flavor : Earth, Oriental / fruity flavor : floral, fruity, cologne
1. type A : black - acidic : Dry / light / high acidity / dark flavors
2. type B : black - bodied : Dry / bold / low acidity / dark flavors
3. type C : cafe latte with sugar - acidic : sweet / bold / high acidity 
4. type D : cafe latte with sugar - bodied : sweet / bold / low acidity 
5. type E : Tea - acidic : Dry / light / high acidity / fruity flavors 
6. type F : Tea - bodied : Dry / bold / low acidity / fruity flavors

## Data Clustering

> 1) Which coffee do you prefer to drink?
- if black or tea: sweetness below 3 -> type a, b, e or f
- if cafe latte w/ sugar: sweetness greater or equal to 3 -> type c or d

> 2) Which type of coffee bean (or tea) do you prefer?
- acidic type: body rate below 3 & acidity greater than 3 -> type a or e
- acidic type: body rate greater than 3 & acidity equal to or lower than 3 -> type c
- full body type: body rate greater than 3 * acidity equal to or lower than 3 -> type b, d or f
    
* dark aroma checked -> type a or b
* fruity aroma checked -> type e or f

### wine_data 

In [41]:
# need more wine data for type_c & d

type_a = df[(df['sweetness'] < 2) & (df['body_rate'] < 3) & (df['acidity'] > 3) & (df['dark_aroma'] == 1)]
type_b = df[(df['sweetness'] < 2) & (df['body_rate'] > 2) & (df['acidity'] <= 3) & (df['dark_aroma'] == 1)]
type_c = df[(df['sweetness'] > 1) & (df['body_rate'] > 3) & (df['acidity'] > 3)]
type_d = df[(df['sweetness'] > 1) & (df['body_rate'] > 3) & (df['acidity'] <= 3)]
type_e = df[(df['sweetness'] < 2) & (df['body_rate'] < 3) & (df['acidity'] > 3) & (df['fruity_aroma'] == 1)]
type_f = df[(df['sweetness'] < 2) & (df['body_rate'] > 2) & (df['acidity'] <= 3) & (df['fruity_aroma'] == 1)]

# add type column
type_a['type'] = 'a'
type_b['type'] = 'b'
type_c['type'] = 'c'
type_d['type'] = 'd'
type_e['type'] = 'e'
type_f['type'] = 'f'

# concat all types into contegrated dataframe
wine_data = pd.concat([type_a, type_b,type_c,type_d,type_e,type_f])
wine_data.tail(2)

,title,image_link,winery,country,province,grape_type,wine_type,alcohol,sweetness,acidity,...,europe_a,europe_b,north_america,south_america,new_world,korea,red,white,wine_id,type
1197,칭퀘테레 코스타 데 캄푸,http://img.wine21.com/WINE_MST/TITLE/0154000/W...,깐티나 칭퀘테레,이탈리아,리구리아,보스코,white,8.5,1.0,3.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1197,f
1198,"깐티나 칭퀘테레, 칭퀘테레",http://img.wine21.com/WINE_MST/TITLE/0154000/W...,깐티나 칭퀘테레,이탈리아,리구리아,보스코,white,8.5,1.0,3.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1198,f


In [42]:
wine_data.tail(2)

,title,image_link,winery,country,province,grape_type,wine_type,alcohol,sweetness,acidity,...,europe_a,europe_b,north_america,south_america,new_world,korea,red,white,wine_id,type
1197,칭퀘테레 코스타 데 캄푸,http://img.wine21.com/WINE_MST/TITLE/0154000/W...,깐티나 칭퀘테레,이탈리아,리구리아,보스코,white,8.5,1.0,3.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1197,f
1198,"깐티나 칭퀘테레, 칭퀘테레",http://img.wine21.com/WINE_MST/TITLE/0154000/W...,깐티나 칭퀘테레,이탈리아,리구리아,보스코,white,8.5,1.0,3.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1198,f


### customer_data

In [44]:
customer_data = pd.DataFrame(columns=['title', 'image_link', 'winery', 'country', 'province',
       'grape_type', 'wine_type', 'alcohol', 'sweetness', 'acidity',
       'body_rate', 'tannin_rate', 'food_pairing', 'price', 'vintage', 'size',
       'link', 'aroma_y', 'meat', 'cheese', 'seafood', 'vegetables', 'dessert',
       'floral', 'fruit', 'citrus', 'oriental_spice', 'oriental_leather',
       'earth', 'dark_aroma', 'fruity_aroma', 'europe_a', 'europe_b',
       'north_america', 'south_america', 'new_world', 'korea', 'red', 'white',
       'wine_id','type'])

# type_a customers
for _ in range(1000):
    customer_data = customer_data.append(wine_data[wine_data['type']=='a'].sample(3), ignore_index=True)
    
# type_b customers
for _ in range(1000):
    customer_data = customer_data.append(wine_data[wine_data['type']=='b'].sample(3), ignore_index=True)

# type_c customers
for _ in range(1000):
    customer_data = customer_data.append(wine_data[wine_data['type']=='c'].sample(3), ignore_index=True)

# type_d customers
for _ in range(1000):
    customer_data = customer_data.append(wine_data[wine_data['type']=='d'].sample(3), ignore_index=True)

# type_e customers
for _ in range(1000):
    customer_data = customer_data.append(wine_data[wine_data['type']=='e'].sample(3), ignore_index=True)

# type_f customers
for _ in range(1000):
    customer_data = customer_data.append(wine_data[wine_data['type']=='f'].sample(3), ignore_index=True)

In [45]:
customer_data.tail(2)

,title,image_link,winery,country,province,grape_type,wine_type,alcohol,sweetness,acidity,...,europe_a,europe_b,north_america,south_america,new_world,korea,red,white,wine_id,type
17998,"메이오미, 샤도네이",http://img.wine21.com/WINE_MST/TITLE/0161000/W...,메이오미,미국,캘리포니아,샤르도네,white,9.5,1.0,3.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,858,f
17999,"베끼아 토레, 샤도네이",http://img.wine21.com/WINE_MST/TITLE/0161000/W...,칸티나 베끼아 토레,이탈리아,뿔리아,샤르도네,white,13.0,1.0,2.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,879,f


### questionaire

In [46]:
question1 = {0 : 'black', 1 : 'milk_sweet', 2: 'tea'}
question2 = {0 : 'acid', 1 : 'body'}
question3 = {0 : 'meat', 1 : 'cheese', 2: 'seafood', 3 : 'vegetables', 4 : 'dessert'}
question5 = 0
# 중복 선택 가능
question4 = {0 : 'floral', 1 : 'fruit', 2 : 'citrus', 3 : 'oriental_spice', 4 : 'oriental_leather', 5: 'earth'} 

In [47]:
questions = [question1, question2, question3, question4, question5]
questions 

[{0: 'black', 1: 'milk_sweet', 2: 'tea'},
 {0: 'acid', 1: 'body'},
 {0: 'meat', 1: 'cheese', 2: 'seafood', 3: 'vegetables', 4: 'dessert'},
 {0: 'floral',
  1: 'fruit',
  2: 'citrus',
  3: 'oriental_spice',
  4: 'oriental_leather',
  5: 'earth'},
 0]

In [48]:
# 설문조사 답
question1_answer = 0
question2_answer = 1
question3_answer = 2
question4_answer = 2, 4
question5_answer = 30000

In [49]:
# 설문조사에 따른 와인 특성 값
question1_result = list(question1.values())[question1_answer]
question2_result = list(question2.values())[question2_answer]
question3_result = list(question3.values())[question3_answer]
question4_result = list(question4.values())[question4_answer[0]], list(question4.values())[question4_answer[1]]
question5_result = question5_answer

In [50]:
# question1, 2(survey) 에 따른 wine_data 나누기

if (question1_result, question2_result) == ('black', 'acid'):
    customer_data = customer_data[customer_data['type'] =='a']
elif (question1_result, question2_result) == ('black', 'body'):
    customer_data = customer_data[customer_data['type'] =='b']
elif (question1_result, question2_result) == ('milk_sweet', 'acid'):
    customer_data = customer_data[customer_data['type'] =='c']
elif (question1_result, question2_result) == ('milk_sweet', 'body'):
    customer_data = customer_data[customer_data['type'] =='d']
elif (question1_result, question2_result) == ('tea', 'acid'):
    customer_data = customer_data[customer_data['type'] =='e']
elif (question1_result, question2_result) == ('tea', 'body'):
    customer_data = customer_data[customer_data['type'] =='f']

customer_data.tail(2)

,title,image_link,winery,country,province,grape_type,wine_type,alcohol,sweetness,acidity,...,europe_a,europe_b,north_america,south_america,new_world,korea,red,white,wine_id,type
5998,"카사 로호, 무쏘 피노 누아",http://img.wine21.com/WINE_MST/TITLE/0163000/W...,카사 로호,스페인,NaN,피노 누아,red,13.0,1.0,3.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,536,b
5999,올드 소울 카버네 소비뇽,http://img.wine21.com/WINE_MST/TITLE/0163000/W...,오크 릿지 와이너리,미국,캘리포니아,카베르네 소비뇽,red,13.0,1.0,2.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,585,b


In [51]:
# question3(food)에 따른 데이터프레임 나누기

if question3_result == 'meat':
    customer_data = customer_data[customer_data['meat'] == 1]
elif question3_result == 'cheese':
    customer_data = customer_data[customer_data['cheese'] == 1]
elif question3_result == 'seafood':
    customer_data = customer_data[customer_data['seafood'] == 1]
elif question3_result == 'vegetables':
    customer_data = customer_data[customer_data['vegatables'] == 1]
elif question3_result == 'dessert':
    customer_data = customer_data[customer_data['dessert'] == 1]
    
customer_data.tail(2)

,title,image_link,winery,country,province,grape_type,wine_type,alcohol,sweetness,acidity,...,europe_a,europe_b,north_america,south_america,new_world,korea,red,white,wine_id,type
5985,"킴 크로포드, 라이즈 앤 샤인 피노 누아",http://img.wine21.com/WINE_MST/TITLE/0161000/W...,킴 크로포드,뉴질랜드,사우스,피노 누아,red,13.5,1.0,3.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,826,b
5993,"몽그라스, 안투 샤도네이",http://img.wine21.com/WINE_MST/TITLE/0165000/W...,비냐 몽그라스,칠레,서던,샤르도네,white,13.5,1.0,3.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,344,b


In [52]:
questions

[{0: 'black', 1: 'milk_sweet', 2: 'tea'},
 {0: 'acid', 1: 'body'},
 {0: 'meat', 1: 'cheese', 2: 'seafood', 3: 'vegetables', 4: 'dessert'},
 {0: 'floral',
  1: 'fruit',
  2: 'citrus',
  3: 'oriental_spice',
  4: 'oriental_leather',
  5: 'earth'},
 0]

In [53]:
# question4(aroma)에 따른 데이터프레임 나누기

if 'floral' in question4_result:
    customer_data = customer_data[customer_data['floral'] == 1]
elif 'fruity' in question4_result:
    customer_data = customer_data[customer_data['fruity'] == 1]
elif 'citrus' in question4_result:
    customer_data = customer_data[customer_data['citrus'] == 1]
elif 'oriental_spice' in question4_result:
    customer_data = customer_data[customer_data['oriental_spice'] == 1]
elif 'oriental_leather' in question4_result:
    customer_data = customer_data[customer_data['oriental_leather'] == 1]
elif 'earth' in question4_result:
    customer_data = customer_data[customer_data['earth'] == 1]

customer_data.tail(2)

,title,image_link,winery,country,province,grape_type,wine_type,alcohol,sweetness,acidity,...,europe_a,europe_b,north_america,south_america,new_world,korea,red,white,wine_id,type
5933,"캠브리아, 케서린 빈야드 샤르도네",http://img.wine21.com/WINE_MST/TITLE/0167000/W...,캠브리아 에스테이트 와이너리,미국,캘리포니아,샤르도네,white,14.5,1.0,3.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,55,b
5948,"캠브리아, 케서린 빈야드 샤르도네",http://img.wine21.com/WINE_MST/TITLE/0167000/W...,캠브리아 에스테이트 와이너리,미국,캘리포니아,샤르도네,white,14.5,1.0,3.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,55,b


In [54]:
# 투표 가자 많이 받은 아이템 sort

customer_data_counts = customer_data['title'].value_counts()
customer_data['counts'] = 0

for i in range(len(customer_data)):
    for n in range(len(customer_data_counts)):
        if customer_data['title'].iloc[i,] == customer_data_counts.index[n]:
            customer_data['counts'].iloc[i,] = customer_data_counts[n]
            
customer_data = customer_data.sort_values(by='counts', ascending=False)
customer_data.drop_duplicates(keep='first', inplace=True)
customer_data

,title,image_link,winery,country,province,grape_type,wine_type,alcohol,sweetness,acidity,...,europe_b,north_america,south_america,new_world,korea,red,white,wine_id,type,counts
3054,칭퀘테레 뻬르골레 스파르세,http://img.wine21.com/WINE_MST/TITLE/0154000/W...,깐티나 칭퀘테레,이탈리아,리구리아,보스코,white,8.5,1.0,3.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1196,b,11
5042,"바타시올로, 랑게 샤르도네 모리노",http://img.wine21.com/WINE_MST/TITLE/0161000/W...,바타시올로,이탈리아,피에몬테,샤르도네,white,8.5,1.0,3.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,864,b,10
4793,클렌드넌 패밀리 빈야드 스탈리온 소비뇽 블랑,http://img.wine21.com/WINE_MST/TITLE/0162000/W...,클렌드넌 패밀리 빈야드,미국,캘리포니아,소비뇽 블랑,white,8.5,1.0,3.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,639,b,9
4848,"코퍼 케인, 보엔 샤르도네",http://img.wine21.com/WINE_MST/TITLE/0164000/W...,코퍼 케인,미국,캘리포니아,샤르도네,white,14.5,1.0,3.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,468,b,9
5948,"캠브리아, 케서린 빈야드 샤르도네",http://img.wine21.com/WINE_MST/TITLE/0167000/W...,캠브리아 에스테이트 와이너리,미국,캘리포니아,샤르도네,white,14.5,1.0,3.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,55,b,8
4497,JR 삿칭 로트기플러,http://img.wine21.com/WINE_MST/TITLE/0161000/W...,요하네스호프 라이니쉬,오스트리아,니더외스터라이히,로트기플러,white,9.0,1.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,917,b,8
5223,"로버트 몬다비, 프라이빗 셀렉션 버본 배럴 샤도네이",http://img.wine21.com/WINE_MST/TITLE/0164000/W...,로버트 몬다비,미국,캘리포니아,샤르도네,white,14.5,1.0,2.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,390,b,8
3318,"안티카, 나파 밸리 샤르도네",http://img.wine21.com/WINE_MST/TITLE/0164000/W...,안티카,미국,캘리포니아,샤르도네,white,14.5,1.0,3.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,500,b,8
4913,라 뷔띠니에르 샤르도네,http://img.wine21.com/WINE_MST/TITLE/0161000/W...,꺄브 안느 드 조와이유스,프랑스,서던,샤르도네,white,9.0,1.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,926,b,8
5001,"깐티나 칭퀘테레, 칭퀘테레",http://img.wine21.com/WINE_MST/TITLE/0154000/W...,깐티나 칭퀘테레,이탈리아,리구리아,보스코,white,8.5,1.0,3.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1198,b,7


In [55]:
# 가격을 무시한 베스트 상품 찾기

best_review = pd.DataFrame(columns=['title', 'alcohol', 'sweetness', 'acidity', 'body_rate', 'tannin_rate', 
                      'meat', 'cheese', 'seafood', 'vegetables', 'dessert', 'floral', 'fruit', 'citrus', 'oriental_spice',
                     'oriental_leather', 'earth', 'europe_a', 'europe_b', 'north_america', 'south_america', 
                      'new_world', 'korea','price', 'link'])

data =list(customer_data.iloc[0,:][['title','alcohol', 'sweetness', 'acidity', 'body_rate', 'tannin_rate', 
                      'meat', 'cheese', 'seafood', 'vegetables', 'dessert', 'floral', 'fruit', 'citrus', 'oriental_spice',
                     'oriental_leather', 'earth', 'europe_a', 'europe_b', 'north_america', 'south_america', 
                      'new_world', 'korea', 'price', 'link']])

data = pd.Series(data, index=best_review.columns)

best_review = best_review.append(data, ignore_index=True)
best_review.set_index('title', drop=False, inplace=True)
best_review

,title,alcohol,sweetness,acidity,body_rate,tannin_rate,meat,cheese,seafood,vegetables,...,oriental_leather,earth,europe_a,europe_b,north_america,south_america,new_world,korea,price,link
title,,,,,,,,,,,,,,,,,,,,,
칭퀘테레 뻬르골레 스파르세,칭퀘테레 뻬르골레 스파르세,8.5,1.0,3.0,4.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,80000.0,https://www.wine21.com/13_search/wine_view.htm...


In [56]:
# price 적용한 데이터프레임

if best_review['price'][0] > question5_answer:
    customer_data_price = customer_data[customer_data['price'] <= question5_answer]
    customer_data_price = customer_data_price[['title', 'alcohol', 'sweetness', 'acidity', 'body_rate', 'tannin_rate', 
                          'meat', 'cheese', 'seafood', 'vegetables', 'dessert', 'floral', 'fruit', 'citrus', 'oriental_spice',
                         'oriental_leather', 'earth', 'europe_a', 'europe_b', 'north_america', 'south_america', 
                          'new_world', 'korea']]
    best_review.drop(['price', 'link'], axis=1, inplace=True)
    best_review = best_review.append(customer_data_price)
    best_review.set_index('title', inplace=True)
    
    # 커피취향, 음식, 아로마, 가격대가 모두 맞고 베스트 상품과 유사도가 가장 높은 와인
    recommend_wine = cosine_similarity(best_review, best_review)
    recommend_wine_df = pd.DataFrame(data=recommend_wine, index = best_review.index, columns=best_review.index)
    recommend_wine_name = recommend_wine_df.iloc[0,:].sort_values(ascending=False)[1:2]
    for i in range(len(customer_data)):
        if customer_data['title'].iloc[i] == recommend_wine_name.index:
            print(customer_data['link'].iloc[i])
            break
    
else:
    recommend_wine_df = best_review
    print(recommend_wine_df['link'])

https://www.wine21.com/13_search/wine_view.html?Idx=164168
